# Emerging Technologies

This notebook contains solutions for the assessment problems on classical and quantum algorithms using the Deutsch–Jozsa algorithm.

## Import Required

In [1]:
import random # problem 1 - random is used for uniform random choices and shuffling;
import numpy as np #  problem 1 - numpy (np) is used later for numeric checks and array operations; 
import itertools as it # problem 1 - itertools (it) is used for generating all binary tuples.


## Introduction

### Classical vs Quantum Information

**[Classical Information](https://en.wikipedia.org/wiki/Classical_information_theory)** - Think about how a CD or DVD stores music and videos. The disc surface contains tiny physical pits — a pit represents a 0, and a flat area (called a "land") represents a 1. When you play the disc, a laser reads these pits one by one, [decoding the binary data sequentially.](https://www.sciencedirect.com/topics/engineering/compact-disc) This is how classical computers work: they process information as definite 0s and 1s, checking each value individually. If you wanted to find out whether a mystery function always gives the same answer (constant) or gives a mix of answers (balanced), you would have to test many inputs one at a time — potentially needing up to 9 queries for a 4-input function.

**[Quantum Information](https://en.wikipedia.org/wiki/Quantum_information)** - Now imagine you need to decide whether to bring a raincoat before going outside. Classically, you would check the weather forecast first, then decide. But what if you could somehow be prepared for all weather possibilities at once? This is the essence of [quantum superposition](https://en.wikipedia.org/wiki/Quantum_superposition) — a **[quantum bit (qubit)](https://en.wikipedia.org/wiki/Qubit)** can exist as both 0 AND 1 simultaneously, like being in a state of "maybe rain, maybe sunshine" until you observe it. The [Deutsch–Jozsa algorithm](https://arxiv.org/abs/quant-ph/9708016) exploits this property: by putting qubits into superposition, we can query a function with all possible inputs at once, and through **[quantum interference](https://en.wikipedia.org/wiki/Wave_interference#Quantum_interference)**, the answer (constant or balanced) reveals itself in just a single query. This is the **[quantum advantage](https://en.wikipedia.org/wiki/Quantum_supremacy)** we will explore in this notebook.


## Problem 1: Generating Random Boolean Functions

**Instructions:**  
Write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of constant or balanced functions taking four **[Boolean](https://en.wikipedia.org/wiki/Boolean_data_type)** inputs.

### 1. Classical Systems and Their State Sets

A classical system is defined by the set of states it can be in.

Examples:

- If \(X\) is a **bit**, then  
Σ = \{0,1\}

- If \(X\) is a **six-sided die**, then  
Σ = \{1,2,3,4,5,6\}


- If \(X\) is a **fan switch**, then  
Σ = {high, medium, low, off}

The physical representation does not matter — only the distinct states matter.

“A bit is just a system that can be in two different states.”

### 2. When We Know the State Exactly

Sometimes we know the state with certainty.

Example:  
If the fan switch is set to *high*, we know the exact classical state.

But in real computation (e.g., networking), we often **don’t** know the state.

### 3. When We Don’t Know the State: Probabilistic States

“If you do any network programming, you’re going to take bits from the network, and you don’t know what they are.”

Real-world information is often uncertain. You cannot predict incoming bits. If you could predict them, the channel would carry less information (because information = unpredictability).

Suppose we believe:

The probability of \(X=0\) is $ \Pr(X=0) = \frac{3}{4} $, and the probability of $X=1$ is $ \Pr(X=1) = \frac{1}{4} $.

This is exactly the IBM Quantum textbook example.

### 4. Representing Uncertainty with Probability Vectors

We represent the probabilistic state as a **[column vector](https://en.wikipedia.org/wiki/Row_and_column_vectors)**:

\[
\begin{bmatrix}
0.75 \\
0.25
\end{bmatrix}
\]

- Top entry = probability of 0  
- Bottom entry = probability of 1  
- They must sum to 1  

A **[probability vector](https://en.wikipedia.org/wiki/Probability_vector)** must satisfy:

1. All entries ≥ 0  
2. Entries sum to 1  

This is the classical analogue of **[quantum state vectors](https://en.wikipedia.org/wiki/Quantum_state#Pure_states_as_rays_in_a_Hilbert_space)**.

### 5. Special Probability Vectors: The Basis States

Two special vectors represent definite classical states:

$$
|0\rangle = \begin{bmatrix}1 \\ 0\end{bmatrix}, \qquad
|1\rangle = \begin{bmatrix}0 \\ 1\end{bmatrix}.
$$

These are the **basis vectors** of classical information. Any probability vector can be written as:

$$
p(0)\,|0\rangle + p(1)\,|1\rangle.
$$

### 6. Demonstrating Probability Vectors in Python




In [9]:
# Example probability vector
p = np.array([[0.75],
              [0.25]])
# Check that the probabilities sum to 1
p.sum()


np.float64(1.0)

### 7. Classical Functions on a Single Bit

A function that takes one bit in and outputs one bit can only behave in **four** possible ways:

| Name | Description | Mapping |
|------|-------------|---------|
| F1 | Constant 0 | 0→0, 1→0 |
| F2 | Identity | 0→0, 1→1 |
| F3 | NOT | 0→1, 1→0 |
| F4 | Constant 1 | 0→1, 1→1 |

“No matter how complicated thecode is, at the end it’s one of these four.”

### 8. Implementing the Four Functions in Python

In [10]:
def f1(a):
    return 0

def f2(a):
    return a

def f3(a):
    return 1 - a

def f4(a):
    return 1

functions = [f1, f2, f3, f4]

### 9. Choosing a Random Function

Now we can select one of these four functions at random. The `random.choice()` function picks one element uniformly from a list—each function has an equal 25% chance of being selected.

**Why this matters:** This simulates receiving an unknown function (like an oracle in quantum computing). We don't know which function we got; we only know it's one of the four possibilities. This uncertainty is central to the Deutsch problem.

In [11]:
# Select a random function from our list
# Each of the four functions has equal probability (1/4 = 25%)
f = random.choice(functions)

# Display the function object
# Note: This doesn't tell us which function it is, just that it exists
print("Selected function:", f)
print("Function name:", f.__name__)

Selected function: <function f3 at 0x7ceb403462a0>
Function name: f3


### 10. Identifying the Random Function

"The only thing we can do is ask the function what happens if we put in 0 and what happens if we put in 1."

**The Classical Approach:**

Since we don't know which function we have, we must **query** it with inputs and observe the outputs. For a single-bit function, there are only two possible inputs: 0 and 1. By testing both inputs, we can determine which of the four functions we have.

This is analogous to treating the function as a **black box** or **oracle**—we can't see inside it, but we can call it and observe what it returns.

In [12]:
# Query the function with both possible inputs
output_0 = f(0)
output_1 = f(1)

print("Query Results:")
print(f"  f(0) = {output_0}")
print(f"  f(1) = {output_1}")
print()

# Show the pattern
print("Output pattern:", (output_0, output_1))
print()
print("Number of queries used: 2")
print("This is the classical baseline - we need 2 queries to fully identify a 1-bit function.")

Query Results:
  f(0) = 1
  f(1) = 0

Output pattern: (1, 0)

Number of queries used: 2
This is the classical baseline - we need 2 queries to fully identify a 1-bit function.


### 11. Determining Which Function It Is

We compare the outputs to the four possibilities. Each unique output pattern corresponds to exactly one function:

| f(0) | f(1) | Function | Type |
|------|------|----------|------|
| 0 | 0 | F1 (constant 0) | Constant |
| 0 | 1 | F2 (identity) | Balanced |
| 1 | 0 | F3 (NOT) | Balanced |
| 1 | 1 | F4 (constant 1) | Constant |

**Self-Learning Insight:**

Notice that we can actually determine if the function is **constant** or **balanced** with just one query in many cases:
- If we're lucky and pick the right input, we might immediately know (e.g., if we only test f(0) and get 0, it could be F1 or F2)
- **But in the worst case**, we need 2 queries to be certain

This is where quantum computing becomes interesting: Can we do better than 2 queries? Deutsch's algorithm answers: **Yes, with quantum superposition, we can determine constant vs. balanced with just 1 quantum query!**

In [13]:
def identify_function(f):
    """
    Identify which of the four single-bit functions f represents.
    
    Args:
        f: A function that takes one bit (0 or 1) and returns one bit
        
    Returns:
        A string describing which function it is
    """
    out0 = f(0)
    out1 = f(1)
    
    if out0 == 0 and out1 == 0:
        return "F1 (constant 0)"
    if out0 == 0 and out1 == 1:
        return "F2 (identity)"
    if out0 == 1 and out1 == 0:
        return "F3 (NOT)"
    if out0 == 1 and out1 == 1:
        return "F4 (constant 1)"

# Identify our randomly chosen function
result = identify_function(f)
print(f"The randomly selected function is: {result}")
print()

# Demonstrate with all four functions to show it works
print("=" * 50)
print("Testing identify_function on all four functions:")
print("=" * 50)
for func in functions:
    identification = identify_function(func)
    print(f"{func.__name__}: {identification}")

The randomly selected function is: F3 (NOT)

Testing identify_function on all four functions:
f1: F1 (constant 0)
f2: F2 (identity)
f3: F3 (NOT)
f4: F4 (constant 1)


**Self-Learning Example: Analysis of Query Complexity**

Let's analyze how many queries we actually need in different scenarios:

In [14]:
# Analysis: Query complexity for different goals

print("SCENARIO 1: Identify the exact function")
print("-" * 40)
print("Worst case: 2 queries (must test both inputs)")
print("Average case: 2 queries (need both to be certain)")
print("Best case: 2 queries (still need both for certainty)")
print()

print("SCENARIO 2: Determine if constant or balanced")
print("-" * 40)
print("Worst case: 2 queries")
print("  Example: Get f(0)=0, could be F1 (constant) or F2 (balanced)")
print("          Need f(1) to distinguish")
print()
print("Best case: 1 query (with luck)")
print("  Example: If both outputs are same, it's constant")
print("          If they differ, it's balanced")
print()

print("QUANTUM ADVANTAGE:")
print("-" * 40)
print("Deutsch's algorithm: 1 quantum query to determine constant vs balanced!")
print("This is why quantum computing is powerful for this problem.")
print()

# Demonstrate random sampling
print("=" * 50)
print("Random Sampling Experiment:")
print("=" * 50)
print("Let's randomly select functions 10 times and see the distribution:")
print()

for i in range(10):
    selected = random.choice(functions)
    func_type = identify_function(selected)
    print(f"Trial {i+1}: {func_type}")

SCENARIO 1: Identify the exact function
----------------------------------------
Worst case: 2 queries (must test both inputs)
Average case: 2 queries (need both to be certain)
Best case: 2 queries (still need both for certainty)

SCENARIO 2: Determine if constant or balanced
----------------------------------------
Worst case: 2 queries
  Example: Get f(0)=0, could be F1 (constant) or F2 (balanced)
          Need f(1) to distinguish

Best case: 1 query (with luck)
  Example: If both outputs are same, it's constant
          If they differ, it's balanced

QUANTUM ADVANTAGE:
----------------------------------------
Deutsch's algorithm: 1 quantum query to determine constant vs balanced!
This is why quantum computing is powerful for this problem.

Random Sampling Experiment:
Let's randomly select functions 10 times and see the distribution:

Trial 1: F1 (constant 0)
Trial 2: F4 (constant 1)
Trial 3: F3 (NOT)
Trial 4: F3 (NOT)
Trial 5: F1 (constant 0)
Trial 6: F4 (constant 1)
Trial 7: F4 (

### 12. Why This Matters for Quantum Computing

Classically, we need **two queries** to fully identify the function.

Quantum computing asks:

> Can we learn something about the function with **one** query?

This leads directly to **[Deutsch's algorithm](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm#Deutsch's_algorithm)**, the first example of **[quantum speedup](https://en.wikipedia.org/wiki/Quantum_computing#Potential)**.

---

### Random Functions (classical toy problem)

One of the simplest toy problems for introducing quantum speedups is to work with raw bits.  
This notebook sets up functions $f:\{0,1\}^n \to \{0,1\}$ that are guaranteed to be either **constant** or **balanced**.  
The $n=3$ example, the random-tuple representation, the combinatorics for balanced functions, and Python code to generate and test random functions are included.

#### Purpose

The problem is reduced to its barest form: functions on bits. No semantic meaning is attached to inputs or outputs — they are raw bits.  
The task: given **[oracle](https://en.wikipedia.org/wiki/Oracle_machine)** access to a function $f$ that is either constant or balanced, determine which type it is. The classical baseline requires many queries; this sets up the Deutsch / Deutsch–Jozsa style question.

#### Definitions

- **[Constant function](https://en.wikipedia.org/wiki/Constant_function):** the output is identical for every input. For a length-$2^n$ tuple the output is either all zeros or all ones.  
- **[Balanced function](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm#Problem_statement):** exactly half of the $2^n$ outputs are $0$ and half are $1$ (so there are $2^{n-1}$ zeros and $2^{n-1}$ ones).

#### Example: $n = 3$ (table representation)

All input triples are listed and one possible function output column is shown. The function column is a length-$2^n$ tuple.

| **a** | **b** | **c** | **f(a,b,c)** |
|-----:|-----:|-----:|:------------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |

- **Constant examples:** $(0,0,0,0,0,0,0,0)$ and $(1,1,1,1,1,1,1,1)$.  
- **Balanced examples:** any tuple with exactly four $0$'s and four $1$'s, e.g. $(0,0,0,0,1,1,1,1)$ or $(1,0,0,1,1,0,0,1)$.

#### Combinatorics for balanced functions ($n=3$)

A balanced function for $n=3$ must have exactly half the outputs $0$ and half $1$, so four $0$'s and four $1$'s in the length-$8$ tuple.  
The number of distinct balanced tuples equals the number of ways to choose which $4$ of the $8$ positions are ones:

$$
\binom{8}{4}
$$

(See **[binomial coefficient](https://en.wikipedia.org/wiki/Binomial_coefficient)**)

This can be computed by counting ordered selections then removing order:

- Ordered selections of 4 distinct positions: $8 \times 7 \times 6 \times 5 = 1680$.
- Divide by the number of orders of those 4 positions, $4! = 24$.

Thus

$$
\binom{8}{4} = \frac{8\cdot7\cdot6\cdot5}{4!} = \frac{1680}{24} = 70.
$$

Therefore there are **70** distinct balanced functions when $n=3$.

#### Random-function model and probabilities

The random model used here is:

1. Choose uniformly between the two types $\{\text{constant},\text{balanced}\}$ (50/50).  
2. If constant: choose uniformly between the two constant tuples (all-0 or all-1).  
3. If balanced: choose uniformly among all $\binom{2^n}{2^{n-1}}$ balanced tuples.

For $n=3$:

- Probability of selecting a particular constant tuple (e.g., all zeros):
  $$
  P(\text{specific constant}) = \tfrac{1}{2}\times\tfrac{1}{2} = \tfrac{1}{4}.
  $$
- Probability of selecting a particular balanced tuple:
  $$
  P(\text{specific balanced}) = \tfrac{1}{2}\times\tfrac{1}{70} = \tfrac{1}{140}.
  $$

---
#### One-bit examples

**Reference:** These represent all possible **[Boolean functions](https://en.wikipedia.org/wiki/Boolean_function)** of one variable. The complete **[truth table](https://en.wikipedia.org/wiki/Truth_table)** enumeration is a fundamental concept in digital logic and the basis for understanding Deutsch's algorithm (see [IBM Quantum Learning: Deutsch-Jozsa Algorithm](https://learning.quantum.ibm.com/course/fundamentals-of-quantum-algorithms/quantum-query-algorithms#the-deutsch-jozsa-algorithm)).

1. Function definitions: four minimal one-bit functions are defined to illustrate constant and non-constant behaviours: f1 and f4 are constant, f2 is identity, f3 is logical NOT.

In [15]:
# One-bit example functions (illustration)
def f1(a):
    return 0 # constant function that always returns 0

def f2(a):
    return a # identity function: returns the input bit unchanged

def f3(a):
    return 0 if a else 1 # NOT function: returns 1 when input is 0, and 0 when input is 1

def f4(a):
    return 1 # constant function that always returns 1


2. random_one_bit_function: demonstrates that functions are **[first-class objects](https://en.wikipedia.org/wiki/First-class_function)** and can be stored in lists; **[random.choice](https://docs.python.org/3/library/random.html#random.choice)** returns one function object using **[uniform random selection](https://en.wikipedia.org/wiki/Discrete_uniform_distribution)**.
3. Test: the chosen function is evaluated on both inputs 0 and 1 to show its behaviour.

In [16]:
# Example: random one-bit function
def random_one_bit_function():
    fns = [f1, f2, f3, f4]  # list of candidate one-bit functions
    return random.choice(fns)  # select and return one function uniformly at random

# Quick test
f = random_one_bit_function()  # obtain a random one-bit function
print("f(0), f(1) =", f(0), f(1))  # evaluate the chosen function on both possible inputs

f(0), f(1) = 1 1


---
#### random_tuple generator
**Purpose**  
Produce a length-$2^n$ tuple representing either a **constant** function (all entries identical) or a **balanced** function (exactly half zeros and half ones).

**Reference:** This implements a **[uniform sampling](https://en.wikipedia.org/wiki/Discrete_uniform_distribution)** strategy over the space of promise functions used in the Deutsch-Jozsa problem. The **[Fisher-Yates shuffle](https://en.wikipedia.org/wiki/Fisher%E2%80%93Yates_shuffle)** (implemented by `random.shuffle`) ensures unbiased random permutation of balanced functions.

* Type selection: ftype is chosen uniformly between 'constant' and 'balanced'.
* Constant branch: zero_or_one picks 0 or 1; (zero_or_one,) * (2**n) constructs the repeated-tuple efficiently.
* Balanced branch: a list is built with the required counts of zeros and ones; random.shuffle randomizes the order; conversion to tuple makes the result hashable/ immutable and consistent with the constant case.
* Notes: using a list for shuffling is necessary because random. shuffle operates in-place on mutable sequences.



In [17]:
def random_tuple(n):
    """Return a random constant or balanced tuple of length 2**n."""
    ftype = random.choice(['constant', 'balanced'])  # choose function type uniformly

    if ftype == 'constant':
        zero_or_one = random.choice([0, 1]) # choose whether the constant is 0 or 1
        t = (zero_or_one,) * (2**n) # create a tuple repeating that bit 2**n times

    else:
        # balanced: create a list with half zeros and half ones, then shuffle
        t = [0] * (2**(n-1)) + [1] * (2**(n-1)) # list with 2^(n-1) zeros followed by 2^(n-1) ones
        random.shuffle(t) # shuffle in-place to randomize positions
        t = tuple(t) # convert list to tuple for immutability/consistency

    return t # return the resulting tuple


---
#### Variadic Functions in Python

**What are Variadic Functions?**

A **[variadic function](https://en.wikipedia.org/wiki/Variadic_function)** is a function that accepts an arbitrary (variable) number of arguments. In languages like **[C](https://en.wikipedia.org/wiki/C_(programming_language))**, you typically must declare the exact number and type of arguments in the function signature (e.g., `int i, char c, char* s`). However, some functions—such as C's famous `printf`—can accept any number of arguments.

**Reference:** The concept of variadic functions dates back to early programming languages and is essential for flexible APIs. C's `printf` function is perhaps the most well-known example, using **[variable argument lists](https://en.wikipedia.org/wiki/Stdarg.h)** (`<stdarg.h>`) to handle formatting strings with any number of values.

**Python's `*args` Syntax**

Python provides syntax for creating variadic functions using the **asterisk operator (`*`)** before an argument name. The most common convention is to name this parameter `*args`, though any valid name can be used.

**How it works:**
- The `*` prefix tells Python to collect all positional arguments into a **[tuple](https://en.wikipedia.org/wiki/Tuple)**
- Inside the function, `args` becomes a tuple containing all the passed arguments
- You can pass any number of arguments (including zero)

**Simple demonstration:**

```python
def function(*args):
    print(args)

function(1, 2, 3, 4, 5)  # Output: (1, 2, 3, 4, 5)
```

**Reference:** This syntax is part of Python's **[argument unpacking](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists)** features, documented in **[PEP 3132](https://www.python.org/dev/peps/pep-3132/)** and extensively covered in resources like **[Real Python's guide to *args and **kwargs](https://realpython.com/python-kwargs-and-args/)**.

---

#### The Explosion (Unpacking) Operator

The asterisk `*` serves **two roles** in Python:

1. **Packing (in function definitions):** `def f(*args)` — collects multiple arguments into one tuple
2. **Unpacking (in function calls):** `f(*my_list)` — expands a list/tuple into separate arguments

This second use is sometimes called the **"explosion operator"** because it "explodes" a collection into individual elements.

**Example:**
```python
def add_three(a, b, c):
    return a + b + c

numbers = [1, 2, 3]
result = add_three(*numbers)  # Same as add_three(1, 2, 3)
```

**Important:** The `*` here is NOT a dereference operator like in C/C++. In Python, it's specifically for argument packing/unpacking.

---

#### Closures and Passing Function Objects

**Closures in Asynchronous Programming**

In JavaScript, when you set up an **[AJAX](https://en.wikipedia.org/wiki/Ajax_(programming))** call to communicate with a network server, the operation is asynchronous—meaning the JavaScript interpreter doesn't stop and wait for the response. Instead, you pass a **function object** to the AJAX call, and once the network request completes, that function will be called automatically. The tricky part is that you're not calling the function yourself from your own code—you're handing the function object over to the AJAX mechanism. This means you can't simply say "here are the arguments I want to pass into that function" at the time you set up the AJAX call.

**Reference:** This callback pattern is fundamental to JavaScript's **[event-driven architecture](https://en.wikipedia.org/wiki/Event-driven_programming)** and asynchronous I/O model (see [MDN Web Docs: Introducing asynchronous JavaScript](https://developer.mozilla.org/en-US/docs/Learn/JavaScript/Asynchronous/Introducing)).

The solution is to use a **[closure](https://en.wikipedia.org/wiki/Closure_(computer_programming))**—you construct a function that already has all the values it needs captured from the surrounding scope. For example, if you have a list of items in JavaScript and you want to run an AJAX call for each one, then put the result into the list item at a specific index (A, B, C, etc.), you typically create a closure that captures that index value. When the AJAX call completes and your callback function runs, it still "remembers" which index it was supposed to update, even though the loop that created it has long since finished executing. This is exactly what closures are for—preserving state across asynchronous boundaries.

**Reference:** The closure captures variables by reference to their lexical scope, a concept rooted in **[lambda calculus](https://en.wikipedia.org/wiki/Lambda_calculus)** and formalized in languages like Scheme before becoming standard in JavaScript (see [Understanding JavaScript Closures](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Closures)).

---

#### Helpers bin_args_to_int and closure fclosure

**Purpose**  
- `bin_args_to_int` converts a binary argument list into an integer index for tuple lookup.  
- `fclosure` creates a **[closure](https://en.wikipedia.org/wiki/Closure_(computer_programming))** that implements a fixed random oracle (constant or balanced) on $n$ bits.

**Reference:** The encoding technique converts **[binary numbers](https://en.wikipedia.org/wiki/Binary_number)** to **[decimal indices](https://en.wikipedia.org/wiki/Positional_notation)** for array lookup—a standard indexing strategy. The closure pattern captures the random tuple in its **[lexical scope](https://en.wikipedia.org/wiki/Scope_(computer_science)#Lexical_scope)**, modeling the "black box" oracle paradigm central to **[query complexity theory](https://en.wikipedia.org/wiki/Query_complexity)** and quantum algorithms (see [Quantum Algorithm Zoo](https://quantumalgorithmzoo.org/)).

**Why `bin_args_to_int` uses `*args`:**

The `bin_args_to_int` function demonstrates variadic functions perfectly:
- It doesn't know in advance how many binary inputs it will receive
- It accepts any number of boolean/truthy values
- The function works for 1 bit, 3 bits, 4 bits, or any $n$ bits
- This flexibility is essential for the quantum oracle abstraction

**How it works:**

1. **Accepts variadic input:** `*args` collects all binary arguments into a tuple
2. **Converts to binary string:** Each argument is evaluated as truthy (→ '1') or falsy (→ '0')
3. **Parses as integer:** The binary string is converted to a decimal index using `int(bits, 2)`
4. **Returns the index:** This index is used to look up the oracle's output in the pre-generated tuple

**Example:**
```python
bin_args_to_int(1, 0, 1)  # Args: (1, 0, 1)
# → bits = '101'
# → int('101', 2) = 5
# → returns 5
```

**Technical notes:**
- Closure pattern: `f_a` is fixed when `fclosure` is called; the returned `f` uses that fixed tuple, modeling an oracle.
- Indexing convention: the first argument corresponds to the most-significant bit in the binary string; maintain this ordering consistently.
- The variadic design allows the same `bin_args_to_int` function to work with any $n$-bit oracle without modification.

In [18]:
def bin_args_to_int(*args):
    """Convert binary args (truthy -> '1', falsy -> '0') to integer index."""
    bits = ''.join('1' if i else '0' for i in args) # build a binary string from positional args
    return int(bits, 2) # parse the binary string as base-2 integer

def fclosure(n):
    """Return a closure f(*args) that implements a random constant or balanced function on n bits."""
    f_a = random_tuple(n)                            # generate the underlying length-2^n tuple once

    def f(*args):
        if len(args) != n:
            return None  # guard: require exactly n input bits
        return f_a[bin_args_to_int(*args)]# map the input bits to an index and return the tuple element

    return f # return the inner function (closure)


---

#### Closures in Python: The `fclosure` Example

**What is a Closure?**

In Python, a **[closure](https://en.wikipedia.org/wiki/Closure_(computer_programming))** is typically a function defined inside another function. This inner function grabs the objects defined in its enclosing scope and associates them with the inner function object itself. The resulting combination is called a closure.

**Reference:** From **[Python Closures: Common Use Cases and Examples](https://realpython.com/inner-functions-what-are-they-good-for/#closures-and-factory-functions)** on Real Python.

**The `fclosure` Function Explained**

The outer function is called `fclosure`. This is the function that returns a randomly selected function `f`. When you call `fclosure`, it returns a function that represents a random constant or balanced oracle. The pattern in the code above works as follows:

1. `f_a = random_tuple(n)` — Generate a random constant or balanced tuple of length $2^n$
2. Define an inner function `f(*args)` that takes n binary arguments
3. The inner function uses `bin_args_to_int(*args)` to convert arguments to an index
4. It returns the element at that index from `f_a`
5. Return the inner function `f` (the closure)

**Why It's Called a Closure**

The tuple `f_a` is created and gets its values in `fclosure` before you ever start talking about the inner function `f`. But notice how `f_a` is used inside the inner function `f`. This is technically interesting because we return `f` at the end of the outer function. 

When you call `fclosure`, it runs and finishes—it doesn't stay alive. Normally, when a function runs and then dies, all the local variables inside die too. You don't expect local variables in a function to hang around after the function has finished running.

**The Key Question: Why Does `f_a` Survive?**

When you call the returned function `f`, it still has access to `f_a` even though `fclosure` has finished executing. It doesn't throw an error saying "I can't find `f_a`". Why? Because everything in Python is an **[object](https://en.wikipedia.org/wiki/Object_(computer_science))**, and objects live on as long as something is pointing to them. This is part of Python's **[garbage collection](https://en.wikipedia.org/wiki/Garbage_collection_(computer_science))** mechanism, similar to Java's model.

**Reference:** Python's execution model and memory management follow the **[reference counting](https://en.wikipedia.org/wiki/Reference_counting)** principle: as long as something is pointing at an object, it doesn't get garbage collected. Only when all references are removed does the object become eligible for cleanup (see [Python Data Model](https://docs.python.org/3/reference/datamodel.html)).

Think of `f_a` as a reference. The inner function `f` is pointing at whatever tuple `f_a` refers to. Python doesn't worry about dereferencing or pointer arithmetic like in C—you don't need the `*` dereference operator or anything like that. Because `f` is pointing at the tuple through `f_a`, and because `f` itself is returned from `fclosure` and kept alive by the caller, the tuple remains accessible. The inner function `f` is now a reference to whatever that tuple is, and `f` keeps a reference to `f_a`, so the garbage collector knows to keep everything around.

**Why the Name "Closure"?**

It's called a closure because the inner function "closes over" variables from the outer function and lives on past the outer function's execution, using values that were originally created in that outer scope. This is a huge idea in programming language theory, particularly in **[functional programming](https://en.wikipedia.org/wiki/Functional_programming)** and languages derived from **[Scheme](https://en.wikipedia.org/wiki/Scheme_(programming_language))** and **[Lisp](https://en.wikipedia.org/wiki/Lisp_(programming_language))**.

**Practical Example:**

```python
# Define a random constant or balanced function taking three binary arguments
f = fclosure(3)

# Try an input on f
f(1, 1, 0)  # Returns 0 or 1 depending on the random tuple

# This doesn't give the game away - you can't see f_a
f  # Shows: <function __main__.fclosure.<locals>.f(*args)>
```

When you look at `f`, it just shows that it's a local function defined inside `fclosure`, but you can't directly access `f_a` to see what the oracle's internal state is. This encapsulation is exactly what we want for modeling a quantum oracle—a black box that we can query but whose internal implementation is hidden.

---
#### try_all and is_constant_or_balanced

**Purpose**  
- `try_all` exhaustively evaluates the oracle on all $2^n$ inputs (classical brute-force).  
- `is_constant_or_balanced` classifies the oracle as `'constant'`, `'balanced'`, or `'neither'` using the exhaustive outputs.

**Reference:** This implements **[brute-force search](https://en.wikipedia.org/wiki/Brute-force_search)** over the input space—the classical baseline requiring $O(2^n)$ queries in the worst case. The **[itertools.product](https://docs.python.org/3/library/itertools.html#itertools.product)** generates the **[Cartesian product](https://en.wikipedia.org/wiki/Cartesian_product)** $(\{0,1\}^n)$ of all binary inputs. This exponential cost is what the Deutsch-Jozsa algorithm reduces to a single quantum query (see [Nielsen & Chuang, "Quantum Computation and Quantum Information", Section 1.4.3](https://en.wikipedia.org/wiki/Quantum_Computation_and_Quantum_Information)).

In [19]:
def try_all(f, n):
    """Try all 2**n binary inputs on f and return the list of outputs."""
    bin_tuples_n = it.product((0,1), repeat=n)  # generator for all binary tuples of length n
    vals = [f(*t) for t in bin_tuples_n] # evaluate f on each tuple and collect outputs
    return vals

def is_constant_or_balanced(f, n):
    """Return 'constant', 'balanced', or 'neither' for function f on n bits (naive test)."""
    results = np.array(try_all(f, n)) # get outputs and convert to numpy array for numeric checks

    # Check if all outputs are 0 or all outputs are 1
    if np.all(results == 0) or np.all(results == 1):
        return 'constant'

    # Check if the number of ones equals 2^(n-1) (balanced)
    elif results.sum() == 2**(n-1):
        return 'balanced'

    # Otherwise the function is neither constant nor balanced (should not occur under the promise model)
    else:
        return 'neither'

# Quick demonstration
f_test = fclosure(3) # create a random function on 3 bits
vals = try_all(f_test, 3) # enumerate outputs for all 8 inputs
print("outputs:", vals) # print the full output tuple
print("classification:", is_constant_or_balanced(f_test, 3))  # print the classification result


outputs: [0, 0, 0, 0, 1, 1, 1, 1]
classification: balanced


### Measuring probabilistic states

**Reference:** This section follows the pedagogical approach from **[IBM Quantum Learning: Measuring Probabilistic States](https://quantum.cloud.ibm.com/learning/en/courses/basics-of-quantum-information/single-systems/classical-information#measuring-probabilistic-states)**, which introduces **[measurement](https://en.wikipedia.org/wiki/Measurement_in_quantum_mechanics)** concepts for classical and quantum systems.

| **Bit** | **Common Labels** | **Truthy/Falsy** | **Ket Notation** | **Column Vector** |
|---------|-------------------|------------------|------------------|-------------------|
| 0       | 0, false, False   | falsy            | $\|0\rangle$     | $\begin{pmatrix}1 \\\\ 0\end{pmatrix}$ |
| 1       | 1, true, True     | truthy           | $\|1\rangle$     | $\begin{pmatrix}0 \\\\ 1\end{pmatrix}$ |

---

A reason **[binary functions](https://en.wikipedia.org/wiki/Boolean_function)** are discussed is that there are many different ways to represent the values **0** and **1**. People often say computers work with zeros and ones, but in practice programming languages expose **[Boolean types](https://en.wikipedia.org/wiki/Boolean_data_type)** with names such as `true` and `false`. The terminology varies, but the underlying idea — two distinct logical values — is the same.

Programming languages also differ in how they treat other values as **[truthy or falsy](https://en.wikipedia.org/wiki/Truthiness)**. For example, an empty list is commonly treated as falsy while a non-empty list is truthy. There is a body of programming-language theory about whether values such as `null` or `None` should be considered true or false. Treating `None` as falsy can hide bugs: a missing reference that should have raised an error may instead behave like `false` and only cause trouble later. For that reason some language designers and programmers prefer to treat `None` (or `null`) as a distinct exceptional case rather than as a simple falsy boolean. In Python, for instance, **[identity checks](https://docs.python.org/3/reference/expressions.html#is)** like `is None` are recommended when the distinction matters.

There are many theoretical and physical ways to represent zeros and ones. A standard mathematical representation uses two-dimensional **[column vectors](https://en.wikipedia.org/wiki/Row_and_column_vectors)** (one column, two rows) to represent the **[basis states](https://en.wikipedia.org/wiki/Basis_(linear_algebra))**. These are commonly written in **[bra–ket notation](https://en.wikipedia.org/wiki/Bra%E2%80%93ket_notation)** as `|0⟩` and `|1⟩`. **[Matrices](https://en.wikipedia.org/wiki/Matrix_(mathematics))** are used because they perform mathematical operations: matrices can be multiplied and manipulated to represent operations on these basis vectors. Matrix and vector representations are extremely useful because **[linear-algebra](https://en.wikipedia.org/wiki/Linear_algebra)** operations implement the transformations that are needed when modelling information and information-processing devices.

A practical example from **[cryptography](https://en.wikipedia.org/wiki/Cryptography)** and **[hashing](https://en.wikipedia.org/wiki/Cryptographic_hash_function)** was mentioned. In **[Bitcoin mining](https://en.wikipedia.org/wiki/Bitcoin#Mining)** and in secure-hash design (for example **[SHA-256](https://en.wikipedia.org/wiki/SHA-2)**), it is desirable that each output bit behaves like an **[unbiased random bit](https://en.wikipedia.org/wiki/Fair_coin)** — roughly a 50/50 distribution for each bit. The unpredictability of each bit prevents easy guessing of hash outputs; a uniform 50/50 distribution across bits is a design goal for cryptographic hash functions.

A pedagogical reference was made to researchers such as **[John Watrous](https://jhwatrous.github.io/)** who use the column-vector / matrix representation extensively. The shorthand ket notation (for example `|0⟩`, `|1⟩`) is convenient: it is easier to write the ket than the corresponding 2×1 column matrix every time. When using this notation it must be clear which matrix is meant by each ket; the ket is a compact label for the underlying vector and for the matrix operations that act on it.

---

#### Measuring a system in a probabilistic state — explanation and examples

Next, consider what happens when a system is **[measured](https://en.wikipedia.org/wiki/Measurement_in_quantum_mechanics)** while it is in a **[probabilistic state](https://en.wikipedia.org/wiki/Probability_distribution)**. Here, *measuring* means looking at the system and recognizing the **[classical state](https://en.wikipedia.org/wiki/Classical_information_theory)** it is in without ambiguity. Intuitively, a probabilistic state cannot be "seen" directly; when the system is **[observed](https://en.wikipedia.org/wiki/Observer_(quantum_physics))**, one sees one of the possible classical states.

When a measurement is made, the observer's knowledge about the system changes. If the measurement reveals that the system is in the classical state `a ∈ Σ`, then the **[probability vector](https://en.wikipedia.org/wiki/Probability_vector)** describing the observer's knowledge becomes the vector with a `1` in the entry for `a` and `0` for all other entries. That vector indicates the system is in the classical state `a` with certainty; this **[standard-basis vector](https://en.wikipedia.org/wiki/Standard_basis)** is denoted `|a⟩` (read "ket a"). Vectors of this form are called **[standard basis vectors](https://en.wikipedia.org/wiki/Standard_basis)**.

**Example (bit basis):** for a single bit the standard basis vectors are `|0⟩` and `|1⟩`, represented by the column vectors (1, 0)ᵀ and (0, 1)ᵀ respectively. Any two-dimensional column vector can be written as a **[linear combination](https://en.wikipedia.org/wiki/Linear_combination)** of these two basis vectors; this generalizes to larger classical state sets.

**Everyday analogy (coin):** suppose a **[fair coin](https://en.wikipedia.org/wiki/Fair_coin)** is flipped and then covered. The probabilistic state of the coin (from the observer's perspective) is "heads with probability 1/2, tails with probability 1/2." If the coin is uncovered and the observer sees tails, the observer updates the description of the coin to the standard basis vector `|tails⟩`. If the coin is covered again and later uncovered by the same observer, the classical state remains tails and the description `|tails⟩` remains correct for that observer.

This update-of-knowledge perspective is important: probabilistic states describe knowledge or belief, not necessarily an **[ontological](https://en.wikipedia.org/wiki/Quantum_mechanics#Philosophical_implications)** superposition of actual classical states. Measuring changes the observer's knowledge. For example, after the coin flip but before looking, the coin is actually either heads or tails — the observer just does not know which. When the observer looks and sees tails, the observer's probability vector becomes `|tails⟩`. To someone who did not see the coin when it was uncovered, the probabilistic state would remain the 50/50 description. Different observers can legitimately have different probability vectors for the same physical system because they have different information. This interpretation is related to **[Bayesian probability](https://en.wikipedia.org/wiki/Bayesian_probability)** and the **[Bayesian interpretation of quantum mechanics](https://en.wikipedia.org/wiki/Quantum_Bayesianism)**.

Although this behaviour is straightforward for classical probabilistic systems, quantum systems behave analogously in the sense that measurement yields a definite classical outcome and updates the observer's description. The difference is that **[quantum amplitudes](https://en.wikipedia.org/wiki/Probability_amplitude)** and **[quantum interference](https://en.wikipedia.org/wiki/Wave_interference#Quantum_interference)** make the underlying mathematics and the allowed transformations richer; establishing the classical analogue first helps make the quantum case less surprising.

---

#### Standard basis and column-vector decomposition

**Reference:** This discussion follows the **[linear algebra](https://en.wikipedia.org/wiki/Linear_algebra)** framework used in quantum information theory, particularly as presented by **[John Watrous](https://jhwatrous.github.io/)** in his work on quantum information (see [The Theory of Quantum Information](https://cs.uwaterloo.ca/~watrous/TQI/)).

These two matrices — the two-row, one-column **[column vectors](https://en.wikipedia.org/wiki/Row_and_column_vectors)** — play a special role. They are called the **[standard basis](https://en.wikipedia.org/wiki/Standard_basis)**. Any two‑by‑one column vector can be written as a **[linear combination](https://en.wikipedia.org/wiki/Linear_combination)** of these basis vectors. Concretely, for a column vector with entries `a` and `b`:

$$
\begin{bmatrix} a \\ b \end{bmatrix}
= a \begin{bmatrix} 1 \\ 0 \end{bmatrix}
+ b \begin{bmatrix} 0 \\ 1 \end{bmatrix}.
$$

When a **[scalar](https://en.wikipedia.org/wiki/Scalar_(mathematics))** is taken outside a matrix, it multiplies the whole matrix; this is standard **[scalar multiplication](https://en.wikipedia.org/wiki/Scalar_multiplication)** of matrices. That property is used in the decomposition above.

These two basis vectors are special, but they are not the only useful vectors. Other vectors can be used as **[basis elements](https://en.wikipedia.org/wiki/Basis_(linear_algebra))** in different contexts, but the standard basis is convenient and widely used.

One practical point to watch for (as emphasised by **[Watrous](https://cs.uwaterloo.ca/~watrous/)** and others) is **[normalization](https://en.wikipedia.org/wiki/Unit_vector#Normalized_vector)** or probability conventions. In some presentations the entries of a **[probability vector](https://en.wikipedia.org/wiki/Probability_vector)** are required to sum to one. When representing a classical probabilistic bit as a column vector, the two entries represent the probabilities of the two classical states (probability of 0 and probability of 1). That probability vector can be written in the standard basis as a **[weighted sum](https://en.wikipedia.org/wiki/Weighted_sum)** of the basis vectors. For example, if the probability of 0 is $p$ and the probability of 1 is $1-p$, then

$$
\begin{bmatrix} p \\ 1-p \end{bmatrix}
= p \begin{bmatrix} 1 \\ 0 \end{bmatrix}
+ (1-p) \begin{bmatrix} 0 \\ 1 \end{bmatrix}.
$$

This expresses the same **[probabilistic model](https://en.wikipedia.org/wiki/Probability_theory)** in the standard-basis vector form.

- The standard-basis vectors make decomposition and bookkeeping explicit: the first **[coordinate](https://en.wikipedia.org/wiki/Coordinate_vector)** multiplies the $|0\rangle$ basis vector and the second coordinate multiplies the $|1\rangle$ basis vector.  
- When a scalar multiplies a basis vector, the scalar scales every entry of that vector; multiplying by zero yields the **[zero-vector](https://en.wikipedia.org/wiki/Zero_element#Linear_algebra)** contribution.  
- The decomposition above is a purely **[algebraic identity](https://en.wikipedia.org/wiki/Identity_(mathematics))** and is valid for any two-component column vector.  
- In probabilistic contexts the entries are **[nonnegative](https://en.wikipedia.org/wiki/Sign_(mathematics)#Real_numbers)** and sum to one; in general linear-algebra contexts they are arbitrary **[scalars](https://en.wikipedia.org/wiki/Scalar_(mathematics))**.

---



### Probability Column Vectors

For quantum stuff, we're gonna need column vectors. We'll use numpy and matplotlib.pyplot as plt.

> Suppose $X$ is a bit. Based on what we know or expect about what has happened to in the past, we might perhaps believe that is in the classical state $0$ with probability $\frac{3}{4}$ and $1$ with probability $\frac{1}{4}$. We may represent these beliefs by... a column vector.
> 
> $ \begin{bmatrix} 0.75 \\ 0.25 \end{bmatrix} $
> 
> *From [Classical states and probability vectors](https://quantum.cloud.ibm.com/learning/en/courses/basics-of-quantum-information/single-systems/classical-information#classical-states-and-probability-vectors) on IBM Quantum Learning.*

So it's a **Probability column vector**.

In [20]:
# The column vector in numpy.
x = np.array([[0.75], [0.25]])

Obviously, to represent them in numpy, we just use an array. 

Now, just be careful, in numpy, one dimensional arrays can be messy. **When you're representing a column vector, you actually have a two dimensional array.**

For example, to represent "Always 0":

In [21]:
# Show.
x

array([[0.75],
       [0.25]])

In [22]:
# Cleaner.
print(x)

[[0.75]
 [0.25]]


---

> We can represent any probabilistic state through a column vector satisfying two properties:
> 
> 1. All entries of the vector are nonnegative real numbers.
> 2. The sum of the entries is equal to 1.
> 
> *From [Classical states and probability vectors](https://quantum.cloud.ibm.com/learning/en/courses/basics-of-quantum-information/single-systems/classical-information#classical-states-and-probability-vectors) on IBM Quantum Learning.*

You can represent any probabilistic state through a column vector. All the entries are non-negative real numbers. It doesn't make any sense to have a probability of minus 0.5.

In [23]:
# Type.
x.dtype

dtype('float64')

In [24]:
# All nonnegative.
# This is going to check for me - numpy.all() goes into a numpy array 
# and checks that all the entries have whatever property.
# So if they're all true, you get true, if any of them are false, then you get false.
# Numpy does operator overloading. This goes into X, which is a numpy array,
# and it asks for each entry in X, is it greater than or equal to 0?
# Technically, this returns a two by one array - it actually returns the boolean array.
np.all(x >= 0.0)

np.True_

In [25]:
# The sum of all probabilities should be 1.
# Whether it's a two dimensional array, a three dimensional, one dimension, 
# all() values if there's no structure involved.
x.sum()

np.float64(1.0)

#### Zero and One

> For example, assuming that the system we have in mind is a bit, the standard basis vectors are given by
> 
> $\ket{0} = \begin{bmatrix} 1 \\ 0 \end{bmatrix}, \qquad \ket{1} = \begin{bmatrix} 0 \\ 1 \end{bmatrix} $
> 
> *From [Measuring probabilistic states](https://quantum.cloud.ibm.com/learning/en/courses/basics-of-quantum-information/single-systems/classical-information#measuring-probabilistic-states) on IBM Quantum Learning.*

In [26]:
######33# Always 0.
zero = np.array([[1], [0]])

print(zero)

[[1]
 [0]]


In [27]:
# Always 1.
one = np.array([[0],[1]])

print(one)

[[0]
 [1]]


In [28]:
# Sometimes call these ket_0 and ket_1.
ket_0 = np.array([[1], [0]])
ket_1 = np.array([[0], [1]])

In [29]:
# ket_0 and ket_1 work well together to represent other states.
ket_x = 0.75 * ket_0 + 0.25 * ket_1

print(ket_x)

[[0.75]
 [0.25]]


#### Matrices

Matrices are two dimensional arrays.

In [30]:
# Matrices as two dimensional arrays.
A = [
  [5, 8, 7],
  [9, 2, 1],
  [4, 6, 3],
]

B = [
  [1, 4, 8],
  [2, 9, 6],
  [3, 5, 7],
]

In [31]:
# Number of rows in A.
len(A)

3

In [32]:
# Number of columns in B.
len(B[0])

3

#### Why Matrix Multiplication is Needed

**[Probability vectors](https://en.wikipedia.org/wiki/Probability_vector)** are combined with matrices to represent operations applied to bits. The matrix represents an operation that transforms the bit's state.

**The workflow:**
- **Start with:** A bit in some state (represented as a probability vector)
- **Apply operation:** Multiply by a matrix (the matrix goes in front)
- **Result:** Get a new state (a new probability vector)

**Real example - the NOT gate:**
- Start: A bit that's 0.75 probability of being 0 and 0.25 probability of being 1
- Apply a **[NOT operation](https://en.wikipedia.org/wiki/Inverter_(logic_gate))** to flip it, even without knowing its exact value
- Result: It becomes 0.25 probability of 0 and 0.75 probability of 1
- The probabilities swap!

**Why this matters:**
- Operations can be applied to bits even when their exact values are unknown
- In quantum computing, bits are often in uncertain states (**[superposition](https://en.wikipedia.org/wiki/Quantum_superposition)**)
- A mathematical way is needed to describe what happens when operations are applied to these uncertain states
- **[Matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication)** provides that tool

---

#### Matrix Multiplication - A Concrete Example

Here's exactly how matrix multiplication works with a real example using two 3×3 matrices 3 rows by 3 columns:

**Matrix A:**
```
[ 5  8  7 ]
[ 9  2  1 ]
[ 4  6  3 ]
```

**Matrix B:**
```
[ 1  4  8 ]
[ 2  9  6 ]
[ 3  5  7 ]
```

**The core concept: Dot Product**

The key operation is the **[dot product](https://en.wikipedia.org/wiki/Dot_product)**:
- Take a row from the first matrix (a list of numbers)
- Take a column from the second matrix (another list of numbers)
- Multiply corresponding entries together
- Add up all those products
- That sum becomes one entry in the result

---

**Step-by-step calculation for position (Row 2, Column 1):**

Calculate one specific entry to show the process:

1. **Identify the row and column:**
   - Target: row 2, column 1 of the result
   - Take **row 2 from matrix A**: [9, 2, 1]
   - Take **column 1 from matrix B**: [1, 2, 3]

2. **Multiply corresponding entries:**
   - First entries: 9 × 1 = 9
   - Second entries: 2 × 2 = 4
   - Third entries: 1 × 3 = 3

3. **Add them all up:**
   - Sum: 9 + 4 + 3 = **16**

4. **Place the result:**
   - The answer 16 goes into **row 2, column 1** of the result matrix

---

**The complete result matrix (A × B):**
```
[ 42  106  121 ]
[ 16   60   90 ]
[ 25   85   85 ]
```

**How all entries are calculated:**
- For **each position** (i, j) in the result matrix:
  - Take **row i** from matrix A
  - Take **column j** from matrix B
  - Do the dot product
  - Put the answer in position (i, j)
- Repeat this 9 times (because 3 rows × 3 columns = 9 entries total)

---

**Important rules:**

- **Dimension compatibility:** The number of columns in the first matrix MUST equal the number of rows in the second matrix
  - Why? Because when doing the dot product, the same number of values is needed in each list
  - Example: A is 3×3, B is 3×3
  - A has 3 columns, B has 3 rows → They match! 
  - The result will be 3×3 (rows from A × columns from B)

- **Order matters:** A × B is usually different from B × A
  - Matrix multiplication is NOT **[commutative](https://en.wikipedia.org/wiki/Commutative_property)**
  - Always be careful about which matrix goes first

- **The process is systematic:**
  - Row from first matrix, column from second matrix
  - Always in that order
  - The position (row, column) in the result corresponds to which row and column were picked

---

**Why this works for quantum computing:**

- When complex systems have numbers that interact in messy ways, matrix multiplication handles it systematically
- It's a universal mathematical tool that captures how **[quantum states](https://en.wikipedia.org/wiki/Quantum_state)** transform
- The matrices represent **[quantum gates](https://en.wikipedia.org/wiki/Quantum_logic_gate)** (operations)
- The vectors represent quantum states
- Multiplying them together shows what happens after the operation
- This framework is why **[linear algebra](https://en.wikipedia.org/wiki/Linear_algebra)** is so central to quantum computing!

#### Non-Commutativity: Why Order Matters

With ordinary numbers, multiplication is **commutative** — we take it for granted that $3 \times 5 = 5 \times 3$. But matrix multiplication is **non-commutative**, meaning:

$$A \times B \neq B \times A \quad \text{(in general)}$$

Reversing the order can produce a completely different result.

**The Socks and Shoes Analogy:**

The best everyday example is getting dressed: *putting on your socks and then your shoes* is **not the same** as *putting on your shoes and then your socks*. The order of operations matters!

This is exactly what non-commutativity means in mathematics. Matrices are probably the first mathematical object most people encounter where this property becomes explicit.

**Why is this significant?**

Non-commutative operations appear everywhere in real-world situations. Historically, physicists working on early **[quantum mechanics](https://en.wikipedia.org/wiki/Quantum_mechanics)** were puzzled by this: multiplying two quantities one way gave a different result than multiplying them the other way. Less than 100 years ago, many in the physics community found this genuinely strange — *"I multiply these two numbers this way and get one result; I multiply them the other way and get a different result!"*

But it turned out that nature itself is non-commutative at the quantum level. There is no deep philosophical reason for it other than: **it works**. In almost every physical situation we try to model, matrix multiplication with its non-commutative property gives us the correct answers.

**Why does swapping the order change the result?**

When you multiply $X \times Y$, you take **rows** from $X$ and **columns** from $Y$. If you swap to $Y \times X$, you are now taking rows from $Y$ and columns from $X$ — you are pairing up completely different sets of numbers. Sometimes you can only multiply them one way: if $X$ is $2 \times 3$ and $Y$ is $3 \times 4$, then $X \times Y$ works (result is $2 \times 4$), but $Y \times X$ does **not** ($Y$ has 4 columns, $X$ has 2 rows — mismatch!).

Let's demonstrate this with our `matrix_multiply` function:

In [2]:
# Multiply.
def matrix_multiply(X, Y):
  """
  Multiply matrices X and Y. Note we should check but don't, for each matrix:
    1. Each row has same number of elements.
    2. Each column has same number of elements.
  Also that:
    3. Number of columns in X equals number of rows in Y.
  """
  return [
    [
        sum([X[i][k] * Y[k][j] for k in range(len(X))])
        for j in range(len(Y[0]))
    ]
    for i in range(len(X))
  ]

In [37]:
# Display it a bit nicer.
for r in matrix_multiply(A, B):
    print('  '.join(f'{x:3}' for x in r))

 42  127  137
 16   59   91
 25   85   89


### Using NumPy

> Unlike in many matrix languages, the product operator `*` operates elementwise in NumPy arrays. The matrix product can be performed using the `@` operator...
> 
> *From [Basic operations](https://numpy.org/doc/stable/user/quickstart.html#basic-operations) in the [NumPy quickstart](https://numpy.org/doc/stable/user/quickstart.html)*

In [38]:
# Matrix multiplication.
np.array(A) @ np.array(B)

array([[ 42, 127, 137],
       [ 16,  59,  91],
       [ 25,  85,  89]])

#### Representing the Four Classical Functions as Matrices

We saw earlier that there are exactly **four** functions that take one bit in and output one bit (f1–f4). Now here is the powerful idea: each of these functions can be represented as a **2×2 matrix**. When we multiply the matrix by a **[probability vector](https://en.wikipedia.org/wiki/Probability_vector)** $\begin{pmatrix} a \\ b \end{pmatrix}$ (where $a + b = 1$), the result is the same as applying the function to that state. This is the framework that looks like **quantum computing** — operations become matrices, states become vectors.

---

**$M_1$ — Constant 0 (f1):**

$$M_1 = \begin{pmatrix} 1 & 1 \\ 0 & 0 \end{pmatrix}$$

Multiplying by a probability vector:

$$\begin{pmatrix} 1 & 1 \\ 0 & 0 \end{pmatrix} \begin{pmatrix} a \\ b \end{pmatrix} = \begin{pmatrix} a + b \\ 0 \end{pmatrix} = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$$

Since $a + b = 1$ for any probability vector, the result is **always** $|0\rangle$ — the bit is definitely 0. When we measure, we get 0 with probability 1. That matrix is equivalent to the constant-0 function!

We can verify with the basis states:
- Input $|0\rangle$: $\begin{pmatrix} 1 & 1 \\ 0 & 0 \end{pmatrix} \begin{pmatrix} 1 \\ 0 \end{pmatrix} = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$ → output 0 ✓
- Input $|1\rangle$: $\begin{pmatrix} 1 & 1 \\ 0 & 0 \end{pmatrix} \begin{pmatrix} 0 \\ 1 \end{pmatrix} = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$ → output 0 ✓

---

**$M_2$ — Identity (f2):**

$$M_2 = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$$

$$\begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} \begin{pmatrix} a \\ b \end{pmatrix} = \begin{pmatrix} a \\ b \end{pmatrix}$$

This always gives back **exactly what you put in**. This special matrix is called the **[identity matrix](https://en.wikipedia.org/wiki/Identity_matrix)** — it is the matrix equivalent of "do nothing." Identity matrices exist for any size: 2×2, 3×3, and so on.

---

**$M_3$ — NOT (f3):**

$$M_3 = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$$

$$\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \begin{pmatrix} a \\ b \end{pmatrix} = \begin{pmatrix} b \\ a \end{pmatrix}$$

**It flips!** The entries swap: if you put in $|1\rangle = \begin{pmatrix} 0 \\ 1 \end{pmatrix}$, you get $\begin{pmatrix} 1 \\ 0 \end{pmatrix} = |0\rangle$. Put one in, you get zero.

An interesting property: **applying NOT twice gives back the identity:**

$$M_3 \times M_3 = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} = M_2$$

Two NOTs cancel out! If you apply the NOT operation twice, you just end up with what you started with. This is why $M_2$ is called the **identity** — it's what you get when an operation undoes itself.

---

**$M_4$ — Constant 1 (f4):**

$$M_4 = \begin{pmatrix} 0 & 0 \\ 1 & 1 \end{pmatrix}$$

$$\begin{pmatrix} 0 & 0 \\ 1 & 1 \end{pmatrix} \begin{pmatrix} a \\ b \end{pmatrix} = \begin{pmatrix} 0 \\ a + b \end{pmatrix} = \begin{pmatrix} 0 \\ 1 \end{pmatrix}$$

Always gives $|1\rangle$ — the opposite of $M_1$.

---

#### Matrices are Associative

While matrices are **not commutative** ($A \times B \neq B \times A$), they **are [associative](https://en.wikipedia.org/wiki/Associative_property)**. This means the **grouping** of multiplications does not matter:

$$(A \times B) \times v = A \times (B \times v)$$

For example, with two NOTs applied to a vector:

$$\left(\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}\right) \begin{pmatrix} a \\ b \end{pmatrix} = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \left(\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \begin{pmatrix} a \\ b \end{pmatrix}\right) = \begin{pmatrix} a \\ b \end{pmatrix}$$

Whether you first multiply the two matrices together and then apply to the vector, or first apply the second matrix to the vector and then the first — the result is the same. The order of the matrices matters (non-commutative), but how you group the multiplications does not (associative).

In [39]:
# Define the four function matrices.
M1 = [[1, 1],  # Constant 0
      [0, 0]]

M2 = [[1, 0],  # Identity
      [0, 1]]

M3 = [[0, 1],  # NOT
      [1, 0]]

M4 = [[0, 0],  # Constant 1
      [1, 1]]

# Define probability vectors (basis states).
ket0 = [[1], [0]]  # |0⟩
ket1 = [[0], [1]]  # |1⟩

# Test M1 (Constant 0): both inputs should give |0⟩.
print("=== M1: Constant 0 ===")
print(f"  M1 × |0⟩ = {matrix_multiply(M1, ket0)} → always |0⟩")
print(f"  M1 × |1⟩ = {matrix_multiply(M1, ket1)} → always |0⟩")

# Test M2 (Identity): output = input.
print("\n=== M2: Identity ===")
print(f"  M2 × |0⟩ = {matrix_multiply(M2, ket0)} → unchanged")
print(f"  M2 × |1⟩ = {matrix_multiply(M2, ket1)} → unchanged")

# Test M3 (NOT): output is flipped.
print("\n=== M3: NOT ===")
print(f"  M3 × |0⟩ = {matrix_multiply(M3, ket0)} → flipped to |1⟩")
print(f"  M3 × |1⟩ = {matrix_multiply(M3, ket1)} → flipped to |0⟩")

# Test M4 (Constant 1): both inputs should give |1⟩.
print("\n=== M4: Constant 1 ===")
print(f"  M4 × |0⟩ = {matrix_multiply(M4, ket0)} → always |1⟩")
print(f"  M4 × |1⟩ = {matrix_multiply(M4, ket1)} → always |1⟩")

=== M1: Constant 0 ===
  M1 × |0⟩ = [[1], [0]] → always |0⟩
  M1 × |1⟩ = [[1], [0]] → always |0⟩

=== M2: Identity ===
  M2 × |0⟩ = [[1], [0]] → unchanged
  M2 × |1⟩ = [[0], [1]] → unchanged

=== M3: NOT ===
  M3 × |0⟩ = [[0], [1]] → flipped to |1⟩
  M3 × |1⟩ = [[1], [0]] → flipped to |0⟩

=== M4: Constant 1 ===
  M4 × |0⟩ = [[0], [1]] → always |1⟩
  M4 × |1⟩ = [[0], [1]] → always |1⟩


In [40]:
# NOT × NOT = Identity?
NOT_squared = matrix_multiply(M3, M3)
print("M3 × M3 (NOT × NOT):")
for row in NOT_squared:
    print(" ", row)
print(f"\nEquals identity M2? {NOT_squared == M2}")
print("Two NOTs cancel out!\n")

# Demonstrate associativity: (M3 × M3) × v  vs  M3 × (M3 × v)
v = [[0.75], [0.25]]  # a general probability vector

# Method 1: multiply matrices first, then apply to vector.
method1 = matrix_multiply(matrix_multiply(M3, M3), v)

# Method 2: apply second matrix to vector first, then first matrix.
method2 = matrix_multiply(M3, matrix_multiply(M3, v))

print("Associativity test with v = [0.75, 0.25]:")
print(f"  (M3 × M3) × v = {method1}")
print(f"  M3 × (M3 × v) = {method2}")
print(f"  Same result? {method1 == method2}")
print("Grouping doesn't matter — matrices are associative!")

M3 × M3 (NOT × NOT):
  [1, 0]
  [0, 1]

Equals identity M2? True
Two NOTs cancel out!

Associativity test with v = [0.75, 0.25]:
  (M3 × M3) × v = [[0.75], [0.25]]
  M3 × (M3 × v) = [[0.75], [0.25]]
  Same result? True
Grouping doesn't matter — matrices are associative!


#### Not Every Matrix Can Represent a Function — Valid Probability Vectors and Stochastic Matrices

We have seen that each of the four classical single-bit functions can be written as a 2×2 matrix. But can **any** 2×2 matrix represent a valid operation on a bit? **No.** There are strict rules about which matrices are allowed and which are not.

---

##### What Makes a Valid Probability Vector?

A **[probability vector](https://en.wikipedia.org/wiki/Probability_vector)** represents the state of a bit. For it to make physical sense as a set of probabilities, it must satisfy **two conditions:**

1. **No negative entries** — probabilities cannot be negative  
2. **Entries must sum to 1** — the total probability across all possible states must be 100%

**Valid examples (OK):**

$$
\begin{pmatrix} 1 \\ 0 \end{pmatrix}, \quad
\begin{pmatrix} 0 \\ 1 \end{pmatrix}, \quad
\begin{pmatrix} 0.75 \\ 0.25 \end{pmatrix}, \quad
\begin{pmatrix} 0.1 \\ 0.9 \end{pmatrix}, \quad
\begin{pmatrix} 0.5 \\ 0.5 \end{pmatrix}
$$

Each of these has all entries ≥ 0, and the entries in each vector add up to exactly 1. These all represent perfectly valid states for a single bit. For instance, $\begin{pmatrix} 0.75 \\ 0.25 \end{pmatrix}$ means "75% chance of being 0 and 25% chance of being 1."

**Invalid examples (NOT OK):**

$$
\begin{pmatrix} 0.7 \\ 0.4 \end{pmatrix}, \quad
\begin{pmatrix} -0.5 \\ 1.5 \end{pmatrix}, \quad
\begin{pmatrix} 1.1 \\ 0 \end{pmatrix}
$$

- $\begin{pmatrix} 0.7 \\ 0.4 \end{pmatrix}$: entries sum to **1.1**, not 1. This says there is a 110% total chance — that makes no sense.
- $\begin{pmatrix} -0.5 \\ 1.5 \end{pmatrix}$: the first entry is **negative**. A probability of −50% is meaningless. (Even though the entries sum to 1, having a negative value violates the first condition.)
- $\begin{pmatrix} 1.1 \\ 0 \end{pmatrix}$: entries sum to **1.1**, not 1. A 110% probability of a single outcome is impossible.

---

##### What Makes a Valid Operation Matrix? — Stochastic Matrices

If we want a matrix to represent a valid probabilistic operation — one that takes a valid probability vector as input and **guarantees** a valid probability vector as output — the matrix must be a **[stochastic matrix](https://en.wikipedia.org/wiki/Stochastic_matrix)**. The key rule is:

> **Each column of the matrix must sum to 1** (and all entries must be ≥ 0).

This is called a **column-stochastic matrix**. Why does this work? When you multiply a stochastic matrix by a probability vector, the output entries are combinations of the input probabilities, weighted by the column entries. If each column sums to 1, the output entries are guaranteed to sum to 1 as well.

**Example — A valid stochastic matrix:**

$$
\begin{pmatrix} 0.5 & 0.5 \\ 0.5 & 0.5 \end{pmatrix} \begin{pmatrix} 0.75 \\ 0.25 \end{pmatrix} = \begin{pmatrix} 0.5 \times 0.75 + 0.5 \times 0.25 \\ 0.5 \times 0.75 + 0.5 \times 0.25 \end{pmatrix} = \begin{pmatrix} 0.5 \\ 0.5 \end{pmatrix}
$$

The output $\begin{pmatrix} 0.5 \\ 0.5 \end{pmatrix}$ sums to **1** ✓ — it is a valid probability vector. Check the columns of the matrix: column 1 is $\begin{pmatrix} 0.5 \\ 0.5 \end{pmatrix}$ (sums to 1 ✓), column 2 is $\begin{pmatrix} 0.5 \\ 0.5 \end{pmatrix}$ (sums to 1 ✓). This matrix represents a "coin flip" operation — no matter what the input state is, the output is always a 50/50 chance.

All four of our function matrices ($M_1$, $M_2$, $M_3$, $M_4$) are stochastic matrices. For instance, the NOT matrix $M_3 = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$ has columns summing to 1: $\begin{pmatrix} 0 \\ 1 \end{pmatrix}$ sums to 1, and $\begin{pmatrix} 1 \\ 0 \end{pmatrix}$ sums to 1. So applying NOT to any valid probability vector will always produce another valid probability vector.

**Example — Breaking the condition (NOT a valid stochastic matrix):**

What happens if we use a matrix whose columns do **not** sum to 1?

$$
\begin{pmatrix} 0.5 & 0.25 \\ 0.3 & 0.5 \end{pmatrix} \begin{pmatrix} 0.75 \\ 0.25 \end{pmatrix} = \begin{pmatrix} 0.5 \times 0.75 + 0.25 \times 0.25 \\ 0.3 \times 0.75 + 0.5 \times 0.25 \end{pmatrix} = \begin{pmatrix} 0.4375 \\ 0.35 \end{pmatrix}
$$

The output entries sum to $0.4375 + 0.35 = 0.7875$ — **not 1!** ✗ This is not a valid probability vector. The matrix broke our input because its columns don't sum to 1 (column 1 sums to 0.8, column 2 sums to 0.75). This matrix **cannot** represent a valid probabilistic operation.

Matrices like $\begin{pmatrix} 1 & 1 \\ 2 & 2 \end{pmatrix}$ are also not allowed — the columns sum to 3, and the entries include values greater than 1. Such a matrix would produce nonsensical "probabilities" as output.

---

##### The Convention: Matrix First, Vector Second

We always write the **operation (matrix) on the left** and the **state (vector) on the right**:

$$\underbrace{\text{(operation)}}_{\text{matrix}} \quad \underbrace{\text{(state)}}_{\text{vector}} = \text{new state}$$

This is a convention used throughout classical probability, quantum computing, and linear algebra. The matrix acts on the vector from the left to produce a new state vector.

In [41]:
# --- Valid vs Invalid Probability Vectors ---
print("=== Valid Probability Vectors (OK) ===")
valid_vectors = [
    [1, 0],
    [0, 1],
    [0.75, 0.25],
    [0.1, 0.9],
    [0.5, 0.5],
]
for v in valid_vectors:
    total = sum(v)
    has_neg = any(x < 0 for x in v)
    print(f"  {v}  sum={total}  negatives={has_neg}  → {'VALID ✓' if total == 1 and not has_neg else 'INVALID ✗'}")

print("\n=== Invalid Probability Vectors (NOT OK) ===")
invalid_vectors = [
    [0.7, 0.4],      # sums to 1.1
    [-0.5, 1.5],     # negative entry
    [1.1, 0],        # sums to 1.1
]
for v in invalid_vectors:
    total = sum(v)
    has_neg = any(x < 0 for x in v)
    reasons = []
    if has_neg:
        reasons.append("has negative")
    if round(total, 10) != 1:
        reasons.append(f"sum={total} ≠ 1")
    print(f"  {v}  → INVALID ✗ ({', '.join(reasons)})")

# --- Stochastic Matrix: columns sum to 1 ---
print("\n=== Stochastic Matrix (valid operation) ===")
stochastic = [[0.5, 0.5],
              [0.5, 0.5]]
input_vec = [[0.75], [0.25]]
result = matrix_multiply(stochastic, input_vec)
result_sum = sum(r[0] for r in result)
print(f"  Matrix columns: col1 sums to {stochastic[0][0] + stochastic[1][0]}, col2 sums to {stochastic[0][1] + stochastic[1][1]}")
print(f"  {stochastic} × {[r[0] for r in input_vec]} = {[r[0] for r in result]}")
print(f"  Output sums to {result_sum} → {'VALID ✓' if result_sum == 1 else 'INVALID ✗'}")

# --- Breaking the condition ---
print("\n=== Non-stochastic Matrix (invalid operation) ===")
bad_matrix = [[0.5, 0.25],
              [0.3, 0.5]]
result_bad = matrix_multiply(bad_matrix, input_vec)
result_bad_sum = sum(r[0] for r in result_bad)
print(f"  Matrix columns: col1 sums to {bad_matrix[0][0] + bad_matrix[1][0]}, col2 sums to {bad_matrix[0][1] + bad_matrix[1][1]}")
print(f"  {bad_matrix} × {[r[0] for r in input_vec]} = {[r[0] for r in result_bad]}")
print(f"  Output sums to {result_bad_sum} → {'VALID ✓' if round(result_bad_sum, 10) == 1 else 'INVALID ✗ (not 1!)'}")

=== Valid Probability Vectors (OK) ===
  [1, 0]  sum=1  negatives=False  → VALID ✓
  [0, 1]  sum=1  negatives=False  → VALID ✓
  [0.75, 0.25]  sum=1.0  negatives=False  → VALID ✓
  [0.1, 0.9]  sum=1.0  negatives=False  → VALID ✓
  [0.5, 0.5]  sum=1.0  negatives=False  → VALID ✓

=== Invalid Probability Vectors (NOT OK) ===
  [0.7, 0.4]  → INVALID ✗ (sum=1.1 ≠ 1)
  [-0.5, 1.5]  → INVALID ✗ (has negative)
  [1.1, 0]  → INVALID ✗ (sum=1.1 ≠ 1)

=== Stochastic Matrix (valid operation) ===
  Matrix columns: col1 sums to 1.0, col2 sums to 1.0
  [[0.5, 0.5], [0.5, 0.5]] × [0.75, 0.25] = [0.5, 0.5]
  Output sums to 1.0 → VALID ✓

=== Non-stochastic Matrix (invalid operation) ===
  Matrix columns: col1 sums to 0.8, col2 sums to 0.75
  [[0.5, 0.25], [0.3, 0.5]] × [0.75, 0.25] = [0.4375, 0.35]
  Output sums to 0.7875 → INVALID ✗ (not 1!)


#### Beyond One Bit — Multiple Systems and the Kronecker Product

Everything we have discussed so far has been about a **single bit**: one system with two possible states (0 or 1), represented by a 2-element probability vector. But real computation — and real quantum computing — involves **multiple bits** working together. How do we describe the combined state of two or more bits?

This is where the **[Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product)** (also called the tensor product) comes in. The symbol is ⊗ — and it is **not the same as regular matrix multiplication**. It is a completely different operation.

---

##### Why Do We Need This?

Suppose you have two separate bits:
- Bit 1 is in state $\begin{pmatrix} a \\ b \end{pmatrix}$ (probability $a$ of being 0, probability $b$ of being 1)
- Bit 2 is in state $\begin{pmatrix} c \\ d \end{pmatrix}$ (probability $c$ of being 0, probability $d$ of being 1)

The combined system of two bits has **four** possible states: 00, 01, 10, and 11. We need a 4-element vector to describe the probabilities of all four outcomes. The Kronecker product gives us exactly this.

---

##### How the Kronecker Product Works — Vectors

**The formula for two 2-element vectors:**

$$\begin{pmatrix} a \\ b \end{pmatrix} \otimes \begin{pmatrix} c \\ d \end{pmatrix} = \begin{pmatrix} ac \\ ad \\ bc \\ bd \end{pmatrix}$$

Each entry in the result corresponds to a two-bit state:

| Result Entry | Two-bit State | Meaning |
|:---:|:---:|:---|
| $ac$ | 00 | Probability both bits are 0 |
| $ad$ | 01 | Probability first bit is 0, second is 1 |
| $bc$ | 10 | Probability first bit is 1, second is 0 |
| $bd$ | 11 | Probability both bits are 1 |

**How to think about it — "take a copy of the second thing":**

Another way to see the same calculation (Option B from lecture notes):

$$\begin{pmatrix} a \begin{pmatrix} c \\ d \end{pmatrix} \\[6pt] b \begin{pmatrix} c \\ d \end{pmatrix} \end{pmatrix} = \begin{pmatrix} ac \\ ad \\ bc \\ bd \end{pmatrix}$$

Take each entry of the **first** vector ($a$, then $b$), and multiply it by a **copy of the entire second vector** $\begin{pmatrix} c \\ d \end{pmatrix}$. Then stack the results on top of each other. This always gives the same answer.

**The sum is still 1:** If both input vectors are valid probability vectors ($a + b = 1$ and $c + d = 1$), then:

$$ac + ad + bc + bd = a(c + d) + b(c + d) = a \cdot 1 + b \cdot 1 = a + b = 1$$

So the Kronecker product of two valid probability vectors is always another valid probability vector. This makes physical sense — the total probability of all possible two-bit outcomes should be 100%.

---

##### Concrete Example

Suppose you have a fair coin (bit 1 is 50/50) and a bit that is definitely 0:

$$\begin{pmatrix} 0.5 \\ 0.5 \end{pmatrix} \otimes \begin{pmatrix} 1 \\ 0 \end{pmatrix} = \begin{pmatrix} 0.5 \times 1 \\ 0.5 \times 0 \\ 0.5 \times 1 \\ 0.5 \times 0 \end{pmatrix} = \begin{pmatrix} 0.5 \\ 0 \\ 0.5 \\ 0 \end{pmatrix}$$

Reading the result: 50% chance of state 00, 0% chance of 01, 50% chance of 10, 0% chance of 11. This makes sense — the second bit is always 0, so only 00 and 10 are possible, each with 50% probability. And the sum is $0.5 + 0 + 0.5 + 0 = 1$ ✓.

The two-digit labels (00, 01, 10, 11) are the **binary representations** of the indices 0, 1, 2, 3. In other words, the bits represent a decimal number: 00 = 0, 01 = 1, 10 = 2, 11 = 3.

---

##### Kronecker Product for Matrices

The Kronecker product also works for matrices — this is how we combine operations on multiple bits. For two 2×2 matrices:

$$\begin{pmatrix} a & b \\ c & d \end{pmatrix} \otimes \begin{pmatrix} e & f \\ g & h \end{pmatrix} = \begin{pmatrix} a \begin{pmatrix} e & f \\ g & h \end{pmatrix} & b \begin{pmatrix} e & f \\ g & h \end{pmatrix} \\[6pt] c \begin{pmatrix} e & f \\ g & h \end{pmatrix} & d \begin{pmatrix} e & f \\ g & h \end{pmatrix} \end{pmatrix} = \begin{pmatrix} ae & af & be & bf \\ ag & ah & bg & bh \\ ce & cf & de & df \\ cg & ch & dg & dh \end{pmatrix}$$

Each entry of the first matrix is replaced by that entry **multiplied by the entire second matrix**. The result is a 4×4 matrix that acts on a 4-element state vector (a two-bit system).

**Why this is useful:** If you want to apply operation $P$ to bit 1 and operation $Q$ to bit 2 simultaneously, the combined operation on the two-bit system is $P \otimes Q$. This framework works for both classical probability and quantum computing.

In [3]:
def kronecker_product_vectors(u, v):
    """
    Compute the Kronecker product of two column vectors.
    Each vector is a list of lists like [[a], [b]].
    Returns a new column vector.
    """
    result = []
    for i in range(len(u)):
        for j in range(len(v)):
            result.append([u[i][0] * v[j][0]])
    return result


def kronecker_product_matrices(A, B):
    """
    Compute the Kronecker product of two matrices.
    Each entry of A is replaced by that entry multiplied by the entire matrix B.
    """
    rows_A, cols_A = len(A), len(A[0])
    rows_B, cols_B = len(B), len(B[0])
    # Result has dimensions (rows_A * rows_B) x (cols_A * cols_B)
    result = [[0] * (cols_A * cols_B) for _ in range(rows_A * rows_B)]
    for i in range(rows_A):
        for j in range(cols_A):
            for k in range(rows_B):
                for l in range(cols_B):
                    result[i * rows_B + k][j * cols_B + l] = A[i][j] * B[k][l]
    return result


# --- Kronecker product of two vectors ---
bit1 = [[0.5], [0.5]]  # Fair coin
bit2 = [[1], [0]]      # Definitely 0

combined = kronecker_product_vectors(bit1, bit2)
labels = ["00", "01", "10", "11"]

print("=== Kronecker Product: Two Vectors ===")
print(f"  Bit 1: {[r[0] for r in bit1]}")
print(f"  Bit 2: {[r[0] for r in bit2]}")
print(f"  Bit 1 ⊗ Bit 2 = {[r[0] for r in combined]}")
print()
for i, label in enumerate(labels):
    print(f"  P({label}) = {combined[i][0]}")
print(f"  Sum = {sum(r[0] for r in combined)} ✓")

# --- Another example ---
print("\n=== Another Example ===")
bit_a = [[0.75], [0.25]]
bit_b = [[0.6], [0.4]]
combined2 = kronecker_product_vectors(bit_a, bit_b)
print(f"  {[r[0] for r in bit_a]} ⊗ {[r[0] for r in bit_b]} = {[round(r[0], 4) for r in combined2]}")
for i, label in enumerate(labels):
    print(f"  P({label}) = {round(combined2[i][0], 4)}")
print(f"  Sum = {sum(r[0] for r in combined2)} ✓")

# --- Kronecker product of two matrices ---
print("\n=== Kronecker Product: Two 2×2 Matrices ===")
print("  Identity ⊗ NOT:")
I = [[1, 0], [0, 1]]   # Identity
NOT = [[0, 1], [1, 0]]  # NOT
I_kron_NOT = kronecker_product_matrices(I, NOT)
for row in I_kron_NOT:
    print(f"    {row}")
print("  This 4×4 matrix applies Identity to bit 1 and NOT to bit 2.")

=== Kronecker Product: Two Vectors ===
  Bit 1: [0.5, 0.5]
  Bit 2: [1, 0]
  Bit 1 ⊗ Bit 2 = [0.5, 0.0, 0.5, 0.0]

  P(00) = 0.5
  P(01) = 0.0
  P(10) = 0.5
  P(11) = 0.0
  Sum = 1.0 ✓

=== Another Example ===
  [0.75, 0.25] ⊗ [0.6, 0.4] = [0.45, 0.3, 0.15, 0.1]
  P(00) = 0.45
  P(01) = 0.3
  P(10) = 0.15
  P(11) = 0.1
  Sum = 1.0 ✓

=== Kronecker Product: Two 2×2 Matrices ===
  Identity ⊗ NOT:
    [0, 1, 0, 0]
    [1, 0, 0, 0]
    [0, 0, 0, 1]
    [0, 0, 1, 0]
  This 4×4 matrix applies Identity to bit 1 and NOT to bit 2.


#### AND and OR Gates as Matrices

With the Kronecker product, we can now represent two-bit inputs as 4-element vectors. The **AND** and **OR** gates become **2×4 matrices** that take a two-bit state in and output a single-bit state.

**AND gate:**

$$\text{AND} = \begin{pmatrix} 1 & 1 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{pmatrix}$$

Each column corresponds to a two-bit input (00, 01, 10, 11). Only input 11 gives output 1 — the last column is $\begin{pmatrix}0\\1\end{pmatrix}$.

**OR gate:**

$$\text{OR} = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 1 & 1 \end{pmatrix}$$

Only input 00 gives output 0 — all other inputs give 1. Note each column still sums to 1, so these are valid stochastic matrices — the output is always a valid probability vector.

In [6]:
# AND gate: 2×4 matrix
AND_gate = np.array([[1, 1, 1, 0],
                     [0, 0, 0, 1]])

# OR gate: 2×4 matrix
OR_gate = np.array([[1, 0, 0, 0],
                    [0, 1, 1, 1]])

# Two-bit inputs as Kronecker products of basis vectors
ket_0 = np.array([[1], [0]])
ket_1 = np.array([[0], [1]])
inputs = {
    "00": np.kron(ket_0, ket_0),
    "01": np.kron(ket_0, ket_1),
    "10": np.kron(ket_1, ket_0),
    "11": np.kron(ket_1, ket_1),
}

print("=== AND Gate ===")
for label, vec in inputs.items():
    result = AND_gate @ vec
    print(f"  AND({label[0]}, {label[1]}): {vec.flatten()} → {result.flatten()}")

print("\n=== OR Gate ===")
for label, vec in inputs.items():
    result = OR_gate @ vec
    print(f"  OR({label[0]}, {label[1]}): {vec.flatten()} → {result.flatten()}")

# Verify columns sum to 1 (stochastic)
print("\n=== Stochastic check (columns sum to 1) ===")
print(f"  AND column sums: {AND_gate.sum(axis=0).tolist()} ")
print(f"  OR  column sums: {OR_gate.sum(axis=0).tolist()} ")

=== AND Gate ===
  AND(0, 0): [1 0 0 0] → [1 0]
  AND(0, 1): [0 1 0 0] → [1 0]
  AND(1, 0): [0 0 1 0] → [1 0]
  AND(1, 1): [0 0 0 1] → [0 1]

=== OR Gate ===
  OR(0, 0): [1 0 0 0] → [1 0]
  OR(0, 1): [0 1 0 0] → [0 1]
  OR(1, 0): [0 0 1 0] → [0 1]
  OR(1, 1): [0 0 0 1] → [0 1]

=== Stochastic check (columns sum to 1) ===
  AND column sums: [1, 1, 1, 1] 
  OR  column sums: [1, 1, 1, 1] 


#### Entanglement: States Cannot be Separated

Here is an idea that connects directly to quantum computing.

Consider this two-bit probability vector:

$$\begin{pmatrix} 0.5 \\ 0 \\ 0 \\ 0.5 \end{pmatrix} \quad \leftarrow \quad \begin{matrix} 00: & 50\% \\ 01: & 0\% \\ 10: & 0\% \\ 11: & 50\% \end{matrix}$$

This state says: "There is a 50% chance both bits are 0 (state 00), and a 50% chance both bits are 1 (state 11). The states 01 and 10 never happen." The sum is $0.5 + 0 + 0 + 0.5 = 1$ ✓ — it is a perfectly valid probability vector.

But here is the puzzling question: **can this state be written as a Kronecker product of two individual single-bit states?**

---

##### Attempting the Decomposition

Let us try. If this state could be separated, there would exist vectors $\begin{pmatrix} a \\ b \end{pmatrix}$ and $\begin{pmatrix} c \\ d \end{pmatrix}$ such that:

$$\begin{pmatrix} a \\ b \end{pmatrix} \otimes \begin{pmatrix} c \\ d \end{pmatrix} = \begin{pmatrix} ac \\ ad \\ bc \\ bd \end{pmatrix} = \begin{pmatrix} 0.5 \\ 0 \\ 0 \\ 0.5 \end{pmatrix}$$

This gives us four equations:
1. $ac = 0.5$ → both $a$ and $c$ must be **non-zero**
2. $ad = 0$ → since $a \neq 0$ (from equation 1), we need $d = 0$
3. $bc = 0$ → since $c \neq 0$ (from equation 1), we need $b = 0$
4. $bd = 0.5$ → but we just showed $b = 0$ and $d = 0$... so $bd = 0 \times 0 = 0 \neq 0.5$

**Contradiction!** At least one number in each of $ad$ and $bc$ has to be zero. But $ac$ and $bd$ being non-zero forces all four values $a$, $b$, $c$, $d$ to be non-zero. There is no solution.

---

##### What Does This Mean?

This particular two-bit state **cannot** be written as the Kronecker product of two independent single-bit states. The two bits are not independent — they are **correlated** in a way that cannot be separated. If you learn that bit 1 is 0, you immediately know bit 2 is also 0 (and vice versa). They are locked together.

In quantum mechanics, this is called **[entanglement](https://en.wikipedia.org/wiki/Quantum_entanglement)** — a correlation between two particles (or qubits) that cannot be described by looking at each particle individually. As your lecturer put it: *"Correlation between two electrons — you can't separate it."*

This is one of the most important and counterintuitive features of quantum computing:

- **Separable states** can be written as $|a\rangle \otimes |b\rangle$ — the two bits are independent
- **Entangled states** **cannot** be decomposed this way — the bits are fundamentally linked
- Entanglement is a key **resource** in quantum algorithms, including the Deutsch–Jozsa algorithm we are studying in this notebook

The state $\begin{pmatrix} 0.5 \\ 0 \\ 0 \\ 0.5 \end{pmatrix}$ is a classical analogue of quantum entanglement — even in classical probability, some joint distributions cannot be factored into independent marginals.

In [7]:
# The "entangled" two-bit state.
entangled = [[0.5], [0], [0], [0.5]]
labels = ["00", "01", "10", "11"]

print("=== The Entangled State ===")
for i, label in enumerate(labels):
    print(f"  P({label}) = {entangled[i][0]}")
print(f"  Sum = {sum(r[0] for r in entangled)} ✓ (valid probability vector)")

# Try every possible combination of single-bit vectors to see if any match.
print("\n=== Can we decompose it as (a, b) ⊗ (c, d)? ===")
# If ac=0.5, ad=0, bc=0, bd=0.5:
# From ad=0: a=0 or d=0
# From bc=0: b=0 or c=0
# But ac=0.5 means a≠0 AND c≠0, so from ad=0 → d=0
# And from bc=0 → b=0 (since c≠0)
# Then bd = 0×0 = 0 ≠ 0.5 → CONTRADICTION!

print("  ac = 0.5 → a ≠ 0 and c ≠ 0")
print("  ad = 0   → since a ≠ 0, d must be 0")
print("  bc = 0   → since c ≠ 0, b must be 0")
print("  bd = 0.5 → but b = 0 and d = 0, so bd = 0 ≠ 0.5")
print("  CONTRADICTION! This state cannot be separated.")

# Show that a separable state CAN be decomposed.
print("\n=== Compare: A Separable State ===")
bit1 = [[0.5], [0.5]]
bit2 = [[1], [0]]
separable = kronecker_product_vectors(bit1, bit2)
print(f"  {[r[0] for r in bit1]} ⊗ {[r[0] for r in bit2]} = {[r[0] for r in separable]}")
for i, label in enumerate(labels):
    print(f"  P({label}) = {separable[i][0]}")
print("  ✓ This state IS separable — the two bits are independent.")

print(f"\n  Entangled state:  {[r[0] for r in entangled]} → NOT separable (bits are correlated)")
print(f"  Separable state:  {[r[0] for r in separable]} → IS separable (bits are independent)")

=== The Entangled State ===
  P(00) = 0.5
  P(01) = 0
  P(10) = 0
  P(11) = 0.5
  Sum = 1.0 ✓ (valid probability vector)

=== Can we decompose it as (a, b) ⊗ (c, d)? ===
  ac = 0.5 → a ≠ 0 and c ≠ 0
  ad = 0   → since a ≠ 0, d must be 0
  bc = 0   → since c ≠ 0, b must be 0
  bd = 0.5 → but b = 0 and d = 0, so bd = 0 ≠ 0.5
  CONTRADICTION! This state cannot be separated.

=== Compare: A Separable State ===
  [0.5, 0.5] ⊗ [1, 0] = [0.5, 0.0, 0.5, 0.0]
  P(00) = 0.5
  P(01) = 0.0
  P(10) = 0.5
  P(11) = 0.0
  ✓ This state IS separable — the two bits are independent.

  Entangled state:  [0.5, 0, 0, 0.5] → NOT separable (bits are correlated)
  Separable state:  [0.5, 0.0, 0.5, 0.0] → IS separable (bits are independent)


### Quantum State Vectors

> *From [Quantum state vectors](https://quantum.cloud.ibm.com/learning/en/courses/basics-of-quantum-information/single-systems/quantum-information#quantum-state-vectors) on IBM Quantum Learning.*

A **quantum state vector** is a column vector whose entries are **complex numbers** ($a + bi$), rather than the real-valued probabilities we used in the classical case. Moving from classical to quantum is just a "twist" — we replace real probabilities with complex amplitudes.

A complex number $a + bi$ (e.g., $2 + 3i$) is just a point on a 2D graph called an **Argand diagram**: the horizontal axis is the real part, the vertical axis is the imaginary part. This is the same idea as Cartesian geometry — just like plotting speed vs time or temperature vs time, except the axes are labelled "real" and "imaginary." A real number like $5$ is simply $5 + 0i$, sitting on the horizontal axis. So real numbers are just a special case of complex numbers, the way the x-axis is part of the whole 2D plane.

---

#### Complex Numbers as 2×2 Matrices (Isomorphism)

Every complex number $a + bi$ can be represented as a 2×2 matrix:

$$a + bi \quad \longleftrightarrow \quad \begin{pmatrix} a & -b \\ b & a \end{pmatrix}$$

This is an **isomorphism** — the matrix behaves identically to the complex number for all arithmetic operations.

**Proving it with multiplication:**

$$(a + bi)(c + di) = ac + adi + bci + bdi^2 = (ac - bd) + (ad + bc)\,i$$

Now multiply the corresponding matrices:

$$\begin{pmatrix} a_0 & -b_0 \\ b_0 & a_0 \end{pmatrix} \begin{pmatrix} a_1 & -b_1 \\ b_1 & a_1 \end{pmatrix} = \begin{pmatrix} a_0 a_1 - b_0 b_1 & -(a_0 b_1 + a_1 b_0) \\ a_0 b_1 + a_1 b_0 & a_0 a_1 - b_0 b_1 \end{pmatrix}$$

Same real and imaginary parts — the isomorphism holds. Note that these special matrices commute with each other ($AB = BA$), but general matrices do not.

---

#### The Complex Conjugate

The **conjugate** of $a + bi$ is $a - bi$ — just flip the sign of the imaginary part. Multiplying a complex number by its conjugate gives:

$$(a + bi)(a - bi) = a^2 - (bi)^2 = a^2 - b^2 i^2 = a^2 + b^2$$

This is always a **real, non-negative number** equal to the **modulus squared** $|z|^2 = a^2 + b^2$. In quantum computing, this matters because measuring a qubit in state $\alpha = a + bi$ gives probability $|\alpha|^2 = a^2 + b^2$.

---

#### The Norm (∥v∥)

The **modulus** of a complex number $a + bi$ is the distance from the origin on the Argand diagram. By Pythagoras (the same right triangle from Cartesian geometry):

$$r = |a + bi| = \sqrt{a^2 + b^2}$$

This generalises to the norm of a quantum state vector with $n$ complex entries:

$$\| v \| = \sqrt{\sum_{k=1}^{n} |\alpha_k|^2}$$

where $|\alpha_k|^2 = a_k^2 + b_k^2$ for $\alpha_k = a_k + b_k i$.

**The quantum rule:** a valid quantum state vector must have $\|v\| = 1$ (compared to the classical rule where entries sum to 1).

---

#### Classical Valid ≠ Quantum Valid

The classical and quantum validity rules are different, so a vector can be valid in one system but not the other:

- $\begin{pmatrix} \frac{1}{4} \\ \frac{3}{4} \end{pmatrix}$ — **valid classical** (entries sum to 1), but **NOT valid quantum**: $\left(\frac{1}{4}\right)^2 + \left(\frac{3}{4}\right)^2 = \frac{1}{16} + \frac{9}{16} = \frac{10}{16} \neq 1$.

- $\begin{pmatrix} \frac{1}{2} \\ \frac{1}{2} \end{pmatrix}$ — **valid classical** (entries sum to 1), but **NOT valid quantum**: $\left(\frac{1}{2}\right)^2 + \left(\frac{1}{2}\right)^2 = \frac{1}{4} + \frac{1}{4} = \frac{1}{2} \neq 1$.

- $\begin{pmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{pmatrix}$ — **NOT valid classical** ($\frac{1}{\sqrt{2}} + \frac{1}{\sqrt{2}} = \sqrt{2} \neq 1$), but **valid quantum**: $\left(\frac{1}{\sqrt{2}}\right)^2 + \left(\frac{1}{\sqrt{2}}\right)^2 = \frac{1}{2} + \frac{1}{2} = 1$ ✓. Note that $\frac{1}{\sqrt{2}}$ is irrational — it cannot be written as a simple fraction.

This last vector is special in quantum computing — it represents a **perfect 50/50 quantum coin**, called the **|+⟩ state**:

$$|+\rangle = \frac{1}{\sqrt{2}}|0\rangle + \frac{1}{\sqrt{2}}|1\rangle$$

When measured, you get 0 or 1 each with probability $\left(\frac{1}{\sqrt{2}}\right)^2 = \frac{1}{2}$.

---

In [8]:
# Problem 1 solution goes here
# Deterministic operations
def random_constant_balanced():
    """
    Returns a randomly chosen constant or balanced function
    that takes 4 Boolean inputs.
    """
    # Step 1: Randomly decide constant or balanced
    is_constant = random.choice([True, False])
    
    if is_constant:
        # Step 2a: Constant function - all 16 outputs are the same
        value = random.choice([0, 1])
        outputs = [value] * 16
    else:
        # Step 2b: Balanced function - 8 zeros and 8 ones, shuffled
        outputs = [0] * 8 + [1] * 8
        random.shuffle(outputs)
    
    # Step 3: Return a function that uses this lookup table
    def f(a, b, c, d):
        # Convert 4 bits to index (0-15)
        index = 8*a + 4*b + 2*c + d
        return outputs[index]
    
    return f

In [9]:
# Test the function
f = random_constant_balanced()

# Print all outputs to see the pattern
print("Input -> Output")
for i in range(16):
    a = (i >> 3) & 1
    b = (i >> 2) & 1
    c = (i >> 1) & 1
    d = i & 1
    print(f"({a},{b},{c},{d}) -> {f(a,b,c,d)}")

# Count outputs to verify constant/balanced
outputs = [f((i>>3)&1, (i>>2)&1, (i>>1)&1, i&1) for i in range(16)]
zeros = outputs.count(0)
ones = outputs.count(1)
print(f"\nZeros: {zeros}, Ones: {ones}")
print(f"Type: {'Constant' if zeros == 0 or zeros == 16 else 'Balanced'}")

Input -> Output
(0,0,0,0) -> 1
(0,0,0,1) -> 0
(0,0,1,0) -> 1
(0,0,1,1) -> 0
(0,1,0,0) -> 1
(0,1,0,1) -> 0
(0,1,1,0) -> 1
(0,1,1,1) -> 0
(1,0,0,0) -> 0
(1,0,0,1) -> 0
(1,0,1,0) -> 1
(1,0,1,1) -> 0
(1,1,0,0) -> 1
(1,1,0,1) -> 1
(1,1,1,0) -> 1
(1,1,1,1) -> 0

Zeros: 8, Ones: 8
Type: Balanced


## Problem 2: Classical Testing for Function Type

**Instructions:**  
Write a Python function `determine_constant_balanced(f)` that analyzes a function f and returns "constant" or "balanced". Include a brief note on efficiency.

In [10]:
# Problem 2 solution goes here

## Problem 3: Quantum Oracles

**Instructions:**  
Using **[Qiskit](https://qiskit.org/)**, create **[quantum oracles](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm#Algorithm)** for each of the possible single-Boolean-input functions in Deutsch's algorithm. Explain how each oracle works.

In [11]:
# Problem 3 solution goes here

## Problem 4: Deutsch's Algorithm with Qiskit

**Instructions:**  
Design a **[quantum circuit](https://en.wikipedia.org/wiki/Quantum_circuit)** that solves Deutsch's problem for single-input functions. Demonstrate its use with each oracle and explain the interference pattern.

In [12]:
# Problem 4 solution goes here

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm

**Instructions:**  
Use Qiskit to create a quantum circuit for four-bit functions generated in Problem 1. Demonstrate it on constant and balanced functions and explain the results.

In [13]:
# Problem 5 solution goes here